# YES 24

In [ ]:
import requests
from bs4 import BeautifulSoup

base_url = "https://www.yes24.com"
data = []

In [ ]:
for i in range(1, 5):
    response = requests.get(
        f"https://www.yes24.com/Product/Category/AttentionNewProduct?pageNumber={i}"
    )
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    books = soup.select(".item_info")

    for book in books:
        # [도서]인 경우만 크롤링
        category_element = book.select_one(".gd_res")
        if not category_element or "[도서]" not in category_element.text:
            continue  # 도서가 아니면 건너뜀

        # 제목
        title_element = book.select_one("a.gd_name")
        title = title_element.text.strip() if title_element else "제목 없음"

        detail_url = base_url + title_element["href"] if title_element else None

        # 저자, 출판사, 출판일 정보
        author_element = book.select_one(
            "div.info_row.info_pubGrp > span.info_auth > a:not(.moreAuthArea a)"
        )
        author = author_element.text.strip() if author_element else "저자 없음"

        publisher_element = book.select_one(
            "div.info_row.info_pubGrp > span:nth-of-type(2)"
        )
        publisher = (
            publisher_element.text.strip() if publisher_element else "출판사 없음"
        )

        pub_date_element = book.select_one(
            "div.info_row.info_pubGrp > span:nth-of-type(3)"
        )
        pub_date = pub_date_element.text.strip() if pub_date_element else "출판일 없음"

        # description 가져오기
        description_element = book.select_one("div.info_row.info_read")
        description = "설명 없음"

        if description_element:
            if detail_url:
                detail_response = requests.get(detail_url)
                detail_soup = BeautifulSoup(detail_response.text, "html.parser")

                # 책 소개 긁어오기
                detail_description = detail_soup.select_one("textarea.txtContentText")
                description = (
                    detail_description.text.strip()
                    if detail_description
                    else "설명 없음"
                )

        print(title, author, publisher, pub_date, description)
        data.append([title, author, publisher, pub_date, description])


#### save

In [ ]:
import pandas as pd

df = pd.DataFrame(data, columns=["title", "author", "publisher", "pub_date", "description"])
df.to_excel("result.xlsx", index=False)